### Exploration Platform

In [1]:
import logging

from utils import ZODHandler as Data
from utils import run_training, WriterHandler

logging.getLogger().setLevel(logging.INFO)

config = {
    "data":{
        "version":"full",
        "ratio":0.003,
        "img_size": 160,
        "transforms":"[]",
        "batch_size":8,
        "test_set":"true"
    },
    "model":{
        "name": "default",
        "args": {
            "num_output":66
        }
    },
    "central":{
        "train":"true",
        "use_gpu":"true",
        "epochs": 3,
        "loss":"MSELoss"
    },
    "federated":{
        "train":"true",
        "global":{
            "n_clients":2,
            "global_rounds":3,
            "client_resources":{
                "num_cpus": 16, 
                "num_gpus": 0
            }
        },
        "client": {
            "epochs": 3
        }
    },
    "swarm":{
        "train":"true",
        "global":{
            "n_clients":10
        },
        "client": {
            "epochs": 3
        }
    }
}

log_dir = WriterHandler(config).writer.log_dir
data = Data(config, log_dir)
#results = run_training(config, data, log_dir)

/home/s0001722/anaconda3/envs/swarm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading val frames: 100%|██████████| 10024/10024 [00:00<00:00, 18589.83it/s]
INFO:root:Training samples: 269, Validation samples: 15, Test samples: 15
INFO:root:Splitting dataset for 2 clients


In [2]:
config = {
    "swarm":{
        "train":"true",
        "global":{
            "n_clients":10
        },
        "client": {
            "epochs": 3
        }
    }
}

sim = SwarmSimulator(config)

In [1]:
import unittest

from utils.training.tests import TestSwarmClient

# Create a test suite from the test case
suite = unittest.TestLoader().loadTestsFromTestCase(TestSwarmClient)

# Create a test runner and run the suite
runner = unittest.TextTestRunner()
runner.run(suite)

/home/s0001722/anaconda3/envs/swarm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-24 00:12:53,883	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(raylet) /home/s0001722/anaconda3/envs/swarm/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet)   aiogrpc.init_grpc_aio()
.2023-02-24 00:13:00,139	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
.2023-02-24 00:13:06,359	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
.2023-02-24 00:13:14,441	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
.2023-02-24 00:13:20,613	INFO worker.py:1529 -- Started a local Ray instance. Vi

<unittest.runner.TextTestResult run=6 errors=0 failures=0>

# Ray Example

In [ ]:
@ray.remote
class Jobs(object):
    def __init__(self):
        self.queue = [*range(1000)]
        self.results = []
    
    def process_the_queue(self):
        self.results.append(self.queue.pop() * 2)
        
    def get_queue(self):
        return self.queue

    def get_results(self):
        return self.results

# Create an actor process.
c = Jobs.remote()

# Check the actor's counter value.
print("QUEUE:", ray.get(c.get_queue.remote()))
print("RESULTS:", ray.get(c.get_results.remote()))

# Increment the counter twice and check the value again.
while len(ray.get(c.get_queue.remote())) > 0:
    c.process_the_queue.remote()

# Check the actor's counter value.
print("QUEUE:", ray.get(c.get_queue.remote()))
print("RESULTS:", ray.get(c.get_results.remote()))

assert len(ray.get(c.get_queue.remote())) in [0, 1000]
assert len(ray.get(c.get_results.remote())) in [0, 1000]

# Other things

In [ ]:
from zod.visualization.oxts_on_image import visualize_oxts_on_image
from zod.constants import Camera, Lidar, Anonymization, Annotatiowidth

# extract the oxts data
oxts = zod_frame.oxts

# visualize the oxts data on the image
calibrations = zod_frame.calibration
from tbparse import SummaryReader
log_dir = "runs"
reader = SummaryReader(log_dir, extra_columns=set(["dir_name"]))
df = reader.scalars


global_metrics = dict(
    centralised = "centralised/loss/epoch",
    federated = "federated/loss/global",
)

reader.scalars.loc[df['tag'].isin(global_metrics.values())]info.keyframe_time.timestamp()

image = zod_frame.get_image(Anonymization.DNAT)
image = visualize_oxts_on_image(oxts, key_timestamp, calibrations, image, camera=Camera.FRONT)

plt.axis("off")
plt.imshow(image)
plt.show()

In [ ]:
from tbparse import SummaryReader
import plotly.express as px
import matplotlib.pyplot as plt

def get_name(s):
    name_chunks = s.split("/")[1].split("_")
    name = name_chunks[0] + " " + name_chunks[-1]
    return name

def get_run(s):
    return s.split("/")[0]

log_dir = "runs"
reader = SummaryReader(log_dir, extra_columns=set(["dir_name"]))
df = reader.scalars


global_metrics = dict(
    centralised = "centralised/loss/epoch",
    federated = "federated/loss",
)

global_results = reader.scalars.loc[df['tag'].isin(global_metrics.values())]

global_results["dir_name"]

# Transformations
global_results["Methods/Sets"] = global_results["dir_name"].apply(get_name)
global_results["Run"] = global_results["dir_name"].apply(get_run)

fig = px.line(
    global_results, 
    x="step", 
    y="value", 
    color="Methods/Sets", 
    width=1280, 
    height=720,
    facet_row="Run")
fig.update_layout(
    template="ggplot2",
    font=dict(family="Helvetica", size=12, color="#404040"),
    title=dict(text="Benchmarks", font=dict(size=24)),
    xaxis=dict(title="Step", showgrid=True),
    yaxis=dict(title="Loss", showgrid=True),
)
fig.show()

In [ ]:
from zod.visualization.oxts_on_image import visualize_oxts_on_image
from zod.constants import Camera, Lidar, Anonymization, Annotatiowidth

# extract the oxts data
oxts = zod_frame.oxts

# visualize the oxts data on the image
calibrations = zod_frame.calibration
from tbparse import SummaryReader
log_dir = "runs"
reader = SummaryReader(log_dir, extra_columns=set(["dir_name"]))
df = reader.scalars


global_metrics = dict(
    centralised = "centralised/loss/epoch",
    federated = "federated/loss/global",
)

reader.scalars.loc[df['tag'].isin(global_metrics.values())]info.keyframe_time.timestamp()

image = zod_frame.get_image(Anonymization.DNAT)
image = visualize_oxts_on_image(oxts, key_timestamp, calibrations, image, camera=Camera.FRONT)

plt.axis("off")
plt.imshow(image)
plt.show()